# **Streaming Ciper**

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from itertools import islice
from functools import reduce
from operator import xor

import sys, os
sys.path.append(os.path.abspath("../"))
from Cangini import LFSR

## Get byte from bit stream

In [2]:
def pseudo_random_byte_generator(seed=None, bit_generator=None, **kwargs):
    
    if bit_generator is None:
        poly = [12, 6, 4, 1, 0]
        bits = LFSR(poly, seed)
    else:
        bits = bit_generator(seed, **kwargs)
        
    while True:
        outp_byte = sum(int(next(bits))<<i for i in range(8))
        yield outp_byte

In [4]:
n_bytes = 100
byte_gen = pseudo_random_byte_generator()

b_stream = [byte for byte in islice(byte_gen, n_bytes)]
#print(key)
bytes(b_stream)[:100]

b'\xff\xd7\xa2\x1aK\xed\xa0\xcb\xd3\xbc\xd8\x7f\xf3S\xfa#C\x82+\x90\xfa\xe7W:\x18<k\x95\x99\x1dE|t\x1b\x90\xcb\xe2\xb96s\x98\xf7f\xd0\n\x95\xfb\x17\x99e\xa2S\xa9,q\x97\x96|_\xf3\x8d\x03h\xb4\xcf\x13\x87\xa7\x96MZ\x1d\x81h\xcc(<Z\x90w\x11.\xd8\xe8\xe8\xd9\xdc2>U\xf1m\x9e\xd7\xc0\x10\x97\xf4v\x83'

## Stream Cipher

In [16]:
class StreamCipher():
    '''
    The class implements the provided Pseudo Random Number Generator.
    
    Methods:
        encrypt: Takes the plaintext end return the encoded ciphertext
        
        decrypt: Takes the ciphertext and return the decoded plaintext
    '''
    
    def __init__(self, key, prng=None, **kwargs):
        if prng is None:
            # If not provided use default PRNG
            self._prng = pseudo_random_byte_generator()
        else:
            # If provided call the function
            self._prng = prng(key, **kwargs)
            
    def encrypt(self, plaintext):
        '''Provide a plaintext as a byte string and return
        the encrypted ciphertext.
        Bytes are encoded byte-to-byte.'''
        return bytes(ch ^ next(self._prng) for ch in plaintext)
    
    def decrypt(self, ciphertext):
        '''Provide a ciphertext as a byte string and return
        the decrypted plaintext.
        Bytes are decoded byte-to-byte.'''
        return bytes(ch ^ next(self._prng) for ch in ciphertext)

### Test the Stream Cipher class with default values:

In [6]:
message = "hello world!"
key = 0x0123456789ABCDEF

alice = StreamCipher(key)
bob = StreamCipher(key)

plainA = message.encode('utf-8')
chipert = alice.encrypt(plainA)
plainB = bob.decrypt(chipert)

print(plainA)
print(chipert)
print(plainB)

b'hello world!'
b'\x97\xb2\xcev$\xcd\xd7\xa4\xa1\xd0\xbc^'
b'hello world!'


## A5/1

In [7]:
class A5_1(object):
    '''
    A5/1 stream cipher
    '''
    
    def __init__(self, key, frame=0, debug=False):
        key_length = 64
        frame_length = 22
        polys = [
            [19, 18, 17, 14, 0],
            [22, 21, 0],
            [23, 22, 21, 8, 0]
        ]
        
        self._lfsrs = [LFSR(poly, state=0) for poly in polys]
        self._ckbits = [10, 11, 12]
        self._count = 0
        
        # Initialize LFSR with key
        if debug:
            print("---- KEY INSERTION ----")
            print("LFSR1  LFSR2   LFSR3")
        for bit, i in zip([int(ci) for ci in f'{key:0{key_length}b}'[::-1]], range(key_length)):
            for lfsr in self._lfsrs:
                lfsr.feedback ^= bit
                next(lfsr)
            if debug and (i < 10):
                print(f'{self._lfsrs[0].state:05x}  {self._lfsrs[1].state:06x}  {self._lfsrs[2].state:06x}')
        if debug:
            print("...    ...     ...")
            print(f'{self._lfsrs[0].state:05x}  {self._lfsrs[1].state:06x}  {self._lfsrs[2].state:06x}')
        
        # Add frame to LFSRs
        if debug:
            print("---- FRAME INSERTION ----")
            print("LFSR1  LFSR2   LFSR3")
        for bit in [int(ci) for ci in f'{frame:0{frame_length}b}'[::-1][:frame_length]]:
            for lfsr in self._lfsrs:
                lfsr.feedback ^= bit
                next(lfsr)
            if debug:
                print(f'{self._lfsrs[0].state:05x}  {self._lfsrs[1].state:06x}  {self._lfsrs[2].state:06x}')
        
        if debug:
            print("---- KEY MIXING ----")
        for _ in range(100):
            next(self)
        
    @property
    def majority(self):
        bits = [bool(lfsr.state & (1 << ckbit)) for lfsr, ckbit in zip(self._lfsrs, self._ckbits)]

        majority = sum(bits) > 1
        return majority
    
    @majority.setter
    def majority(self, val):
        raise AttributeError('Denied')
    
    def __iter__(self):
        return self
    
    def __next__(self):
        maj = self.majority
        
        for lfsr, ckbit in zip(self._lfsrs, self._ckbits):
            if bool(lfsr.state & (1<<ckbit)) == maj:
                next(lfsr)
        
        return self.output
    
    # ==== OUTPUT PROPERTY ====
    @property
    def output(self):
        output = reduce(xor, [lfsr.output for lfsr in self._lfsrs])
        return output
    @output.setter
    def output(self, val):
        raise AttributeError('Denied')
    
    # ==== PRINT CURRENT REGISTER STATUS ===
    def __str__(self):
        _str = f'LFSR1: {self._lfsrs[0].state:05x}  LFSR2: {self._lfsrs[1].state:06x}  LFSR3: {self._lfsrs[2].state:06x}  output: {self.output}'
        return _str
    def __repr__(self):
        return f'A5/1 PRNG -> ({str(self)})'

In [8]:
key, frame = 0x0123456789ABCDEF, 0x2F695A
a5_1 = A5_1(key, frame, True)
print(a5_1)

---- KEY INSERTION ----
LFSR1  LFSR2   LFSR3
40000  200000  400000
60000  300000  600000
70000  380000  700000
78000  3c0000  780000
3c000  1e0000  3c0000
5e000  2f0000  5e0000
6f000  378000  6f0000
77800  3bc000  778000
7bc00  3de000  3bc000
3de00  1ef000  5de000
...    ...     ...
1cc07  19f655  07820a
---- FRAME INSERTION ----
LFSR1  LFSR2   LFSR3
4e603  2cfb2a  03c105
67301  167d95  01e082
73980  2b3eca  00f041
79cc0  159f65  407820
7ce60  0acfb2  603c10
7e730  2567d9  301e08
3f398  12b3ec  580f04
1f9cc  0959f6  6c0782
0fce6  04acfb  3603c1
47e73  02567d  5b01e0
63f39  212b3e  2d80f0
71f9c  10959f  16c078
78fce  084acf  4b603c
7c7e7  242567  25b01e
7e3f3  3212b3  12d80f
7f1f9  190959  096c07
7f8fc  0c84ac  04b603
7fc7e  264256  025b01
3fe3f  13212b  012d80
5ff1f  299095  4096c0
6ff8f  34c84a  604b60
37fc7  1a6425  7025b0
---- KEY MIXING ----
LFSR1: 45447  LFSR2: 0cfd41  LFSR3: 2a90e7  output: 1


In [9]:
message = "hello world!"
key = 0x0123456789ABCDEF
frame = 0x2F695A

alice = StreamCipher(key, bit_generator=A5_1, frame=frame)
bob = StreamCipher(key, bit_generator=A5_1, frame=frame)

plainA = message.encode('utf-8')
chipert = alice.encrypt(plainA)
plainB = bob.decrypt(chipert)

print(plainA)
print(chipert)
print(plainB)

b'hello world!'
b'\x97\xb2\xcev$\xcd\xd7\xa4\xa1\xd0\xbc^'
b'hello world!'


## Rivest Cipher 4 (RC4)

In [12]:
class RC4(object):
    '''
    RC4 Stream Cipher implementation
    '''
    
    def __init__(self, key, key_length=None, drop=0):
        '''
        
        '''
        
        self._L = key_length # Store the key length
        if self._L is None:
            self._L = len(key)
        
        self._i = 0
        self._j = 0
        
        # Generate identity permutation
        self._P = np.array(range(256))
        # Initialize permutation with given key
        for i in range(256):
            self._j = (self._j + self._P[i] + key[i % self._L]) % 256
            self._P[i], self._P[self._j] = self._P[self._j], self._P[i]
        
        self._j = 0
        
        # If drop > 0, discard initial #drop bytes produced
        for _ in range(drop):
            next(self)
    
    def __iter__(self):
        return self
    
    def __next__(self):
        self._i = (self._i + 1) % 256
        self._j = (self._j + self._P[self._i]) % 256
        self._P[self._i], self._P[self._j] = self._P[self._j], self._P[self._i]
        return self._P[(self._P[self._i] + self._P[self._j]) % 256]

### Test the RC4 implementation
Implements the code via the StreamCipher class already developed

In [19]:
message = "hello world!"
key = b'0123456789ABCDEF'
n_drop = 3072

alice = StreamCipher(key, prng=RC4, drop=n_drop)
bob = StreamCipher(key, prng=RC4, drop=n_drop)

plainA = message.encode('utf-8')
chipert = alice.encrypt(plainA)
plainB = bob.decrypt(chipert)

print(plainA)
print(chipert)
print(plainB)

b'hello world!'
b'/\x9e\xf9\x83@\x81}\xa9\xd0\xd4\xd5\xf4'
b'hello world!'
